In [ ]:
# Author:songxindong
# Created: 2025-10-15
# Last Modified: 2025-10-15 14:27:33
# 技术文档: https://km.sankuai.com/collabpage/2727150323

In [2]:
dt = '20250810'

In [3]:
!pip install openai
!pip install pyarrow
!pip install fsspec
!pip install openpyxl

Looking in indexes: https://pip.sankuai.com/simple/, https://pypi.sankuai.com/simple/
Looking in indexes: https://pip.sankuai.com/simple/, https://pypi.sankuai.com/simple/
Looking in indexes: https://pip.sankuai.com/simple/, https://pypi.sankuai.com/simple/
Looking in indexes: https://pip.sankuai.com/simple/, https://pypi.sankuai.com/simple/


In [4]:
import openai
from openai import OpenAI
import pandas as pd
from collections import defaultdict
import sys
import os
import pyarrow

In [ ]:
os.environ['JUPYTER_CONDA_ARCHIVE'] = 'viewfs:///user/hadoop-caterb2b-mall-risk-management/jupyter/4f30bed3-2cae-400e-b76e-3339664503d3/archive/codelab-8e4ce7edc4a6eb36eb5ec2d7b13d2276.zip'
os.environ['PYSPARK_DRIVER_PYTHON'] = '//conda/envs/codelab/bin/python'
os.environ['PYSPARK_PYTHON'] = './ARCHIVE/conda/envs/codelab/bin/python'
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--conf spark.executor.cores=2 '
    '--conf spark.kubernetes.memoryOverheadFactor=0.1 '
    '--conf spark.executor.memory=4g '
    'pyspark-shell'
)

In [6]:
%spark_version 3.x

In [ ]:
%%spark 
--conf spark.yarn.queue=root.zw05_training_cluster.hadoop-caterb2b.training
--conf spark.yarn.dist.archives=viewfs:///user/hadoop-caterb2b-mall-risk-management/jupyter/4f30bed3-2cae-400e-b76e-3339664503d3/archive/codelab-8e4ce7edc4a6eb36eb5ec2d7b13d2276.zip#ARCHIVE
--conf spark.pyspark.python=./ARCHIVE/conda/envs/codelab/bin/python
--conf spark.pyspark.driver.python=./ARCHIVE/conda/envs/codelab/bin/python
--conf spark.yarn.appMasterEnv.PYSPARK_PYTHON=./ARCHIVE/conda/envs/codelab/bin/python

,Driver日志,队列信息
,/home/sankuai/logs/spark-1761556766.log,root.zw05_training_cluster.hadoop-caterb2b.training


CmdOutput(['tail', '-50', '/home/sankuai/logs/spark-1761556766.log'], 2)

Setting spark.hadoop.yarn.resourcemanager.principal to hadoop-caterb2b-mall-risk-management
SLA info url: http://cantor.data.sankuai.com/api/v1/task/normal_spark_songxindong_7f22a9bb/level
SLA info result: {
  "data": {
    "level": 0, 
    "taskWeight": ""
  }, 
  "success": true
}

Set spark.shuffle.manager=rss because spark.k8s.app.tags.tasktype is ml.
Set spark.rss.hdfs.nameSpace to value : hdfs://zw06-rss-mlprod04/zw06rssmlprod04nn
spark.rss.hdfs.base.nn.index : 3
Set spark.sql.adaptive.localShuffleReader.enabled=false, because rss can not read from local and it may also result read amplification
getExtraConf info url: http://themis.data.sankuai.com/api/spark/whitelist/config/etl/normal_spark_songxindong_7f22a9bb
getExtraConf info result: {
  "success" : false,
  "appName" : "normal_spark_songxindong_7f22a9bb",
  "arguments" : null,
  "enabled" : false
}
set spark.reducer.maxReqsInFlight to value: 10
set spark.reducer.maxBlocksInFlightPerAddress to value : 1
Invalid cantor schedul

,SparkSession,SparkContext,Job Search/applicationId
链接/变量名,spark,sc,spark-application-1761556787455


In [8]:
import datetime
etl_dt = dt
friday_dt = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
print(etl_dt)

20250810


In [9]:
%%sql sdf  --preview --quiet
SELECT DISTINCT
       customer_id,
       customer_name,
       contact_id,
       bd_id,
       bd_name,
       area_name,
       internet_call_type,
       talking_start_time,
       end_time,
       talking_time_len,
       bd_tel_num,
       customer_tel_num,
       record_url,
       voice_info,
       processed_sentences
From mart_caterb2b_mall_risk_management.telemarketing_call_info
Where dt = '{etl_dt}'

,customer_id,customer_name,contact_id,bd_id,bd_name,area_name,internet_call_type,talking_start_time,end_time,talking_time_len,bd_tel_num,customer_tel_num,record_url,voice_info,processed_sentences
0,25609602,杂粮煎饼·烤冷面（海关学院店）,1xxxx38dd5d83909258a57eb316260936ed16b4,32436184,邓西何,电销一区,2,2025-08-10 18:22:49,2025-08-10 18:23:31,42000,oMnTEDsEgfDq+EqWQ1ouaA==,aTz6yop9T7l0p6YxJVfpWw==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx38dd5d83909258a57eb31626...","[""bd: 喂哎姐你好，我这边的话是快驴进货的业务经理，就是呃昨天您这边买的那些菜品吗？今天..."
1,26734551,叫花鸡,1xxxx385759207d3a60bbea6c1c1a3bc922f0e6,32362406,胡尧,电销一区,2,2025-08-10 14:10:06,2025-08-10 14:11:10,64000,tK9el9VyOWhSE4xm5e+HAA==,xz6gDShPbqqNVX9Mz3fPmA==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx385759207d3a60bbea6c1c1a...","[""客户: 嗯。"", ""bd: 喂哥。"", ""客户: 哎喂，你好喂，你好。"", ""bd: 哎..."
2,26736292,炒粉快餐,1xxxx3890307ab50c6a21e0df94bd464ff55068,32197355,王绍海,电销一区,2,2025-08-10 21:48:20,2025-08-10 21:49:19,59000,ws4yIiw9s8C9IKYRoxxzcg==,bgUPbJqDmW+QuhULTkOtCA==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx3890307ab50c6a21e0df94bd...","[""bd: 喂。"", ""客户: 喂你好。"", ""bd: 哎姐姐你好，我是那个快驴的姐。"", ..."
3,26733023,鹤祥宫,1xxxx3836162c83dbf3b498cb81f2b106c12b6d,32317138,刘海美,电销一区,2,2025-08-10 19:50:59,2025-08-10 19:53:28,149000,iNVXfaGwtWe+WVWc4cXXiw==,M0Gik2c6crMPiBJWrH/UTA==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx3836162c83dbf3b498cb81f2...","[""客户: 哎喂。"", ""bd: 喂，哎老板您好，您这边是鹤祥宫店的老板吗？"", ""客户: ..."
4,26722122,土耳其烤肉拌饭（千灯古镇店）,1xxxx3864487eee806faadd530a3930c3ea4d95,32197355,王绍海,电销一区,2,2025-08-10 14:07:43,2025-08-10 14:08:47,64000,ws4yIiw9s8C9IKYRoxxzcg==,xawfDNgys1t/9FaTQvUIgg==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx3864487eee806faadd530a39...","[""客户: 哎。"", ""bd: 哎喂，大哥。"", ""bd: 喂哎师傅，我那个快驴的，就是你说..."
5,26731877,小友大锅卤菜,1xxxx38d2b2d80b0357d9dd9d5ee52a287805be,32331470,吴琴梅,电销一区,2,2025-08-10 17:27:38,2025-08-10 17:28:48,70000,IZh53OWrWjibiZWH94IvYQ==,ITpiQX/nM3czVtBSnRIDDQ==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx38d2b2d80b0357d9dd9d5ee5...","[""客户: 喂。"", ""bd: 哎喂，姐你好，哎我快驴的那个小吴，就是我问一下，咱们今天给您..."
6,26672323,沂蒙山炒鸡,1xxxx38d14f61cf3b22a09d0a4e0f2a74644ee8,32197355,王绍海,电销一区,2,2025-08-10 14:17:27,2025-08-10 14:18:23,56000,ws4yIiw9s8C9IKYRoxxzcg==,1FQYqNw0UlFyGu4gDVxfsw==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx38d14f61cf3b22a09d0a4e0f...","[""客户: 喂，你告诉江德福她有怀孕就是把枪。"", ""bd: 喂喂喂喂，大哥。"", ""客户:..."
7,26727124,网鱼电竞（太仓丽景商业广场店）,1xxxx389a0134bcdd25092bdf6f3d52699f373c,32192262,苟玛她姆,电销一区,2,2025-08-10 20:47:31,2025-08-10 20:48:39,68000,q6fyeYX8yMprHIpMJkCvIA==,307NhAAtO0y0JS0/h2kjWQ==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx389a0134bcdd25092bdf6f3d...","[""客户: 喂你好。"", ""bd: 喂哎哥，我小王就是我说你今天有啥有啥东西缺了，不，今天下..."
8,26700462,鼎鲜潮牛坊,1xxxx38ecab8302d9856b34fd678602e2a6a45f,32334766,王绍芳,电销一区,2,2025-08-10 21:55:18,2025-08-10 21:56:01,43000,wgnPK+36i8d5eZlpHuKfxg==,idUeCpUO8KzdXm/o5MGnOA==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx38ecab8302d9856b34fd6786...","[""bd: 喂你好。"", ""客户: 你好。"", ""bd: 哎晚上好，哥，那个生意兴隆，我是那..."
9,26734310,天津全麦烙饼（天北市场早市地下店）,1xxxx38defd376ee5f065a27a06183fada6aa63,32424174,张鹏,电销一区,2,2025-08-10 11:10:34,2025-08-10 11:12:29,115000,8tbgQJbZK2+exD4Hmnp0iw==,8bW4DjkTexQUwB0oXEFr2g==,http://s3plus.vip.sankuai.com/v1/mss_756f88058...,"{""contactId"":""11_1xxxx38defd376ee5f065a27a0618...","[""客户: 哎。"", ""bd: 哎喂，你好，请问是天津全麦烙饼的老板吗？"", ""客户: 啊。..."


In [ ]:
sdf.createOrReplaceTempView('call_info_process')

In [ ]:
# -*- coding: utf-8 -*-
import json
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

# ---------- 1. 读表 ----------
sdf = spark.table("call_info_process")
# ---------- 2. 获取processed_sentences、contact_id ----------
df_conv = sdf.select(
    col("processed_sentences"),
    col("contact_id")                  
)
# ---------- 3. 获取 prompt ----------
prompt_path = "viewfs://hadoop-meituan/user/hadoop-caterb2b-mall-risk-management/rcc/prod/data/prompt/call_info_risk_identify_prompt.txt"
# 从指定路径读取文本文件内容，并合并为单个字符串
prompt_content = "\n".join(  # 使用换行符连接列表中的所有字符串元素
    spark.read.text(prompt_path)  # 使用Spark读取指定路径的文本文件，返回DataFrame
        .rdd  # 将DataFrame转换为RDD（弹性分布式数据集）以进行分布式操作
        .map(lambda r: r.value)  # 使用map操作提取RDD中每行的value字段（即文本内容）
        .collect()  # 将分布式RDD数据收集到驱动节点，转换为本地列表
)

# ---------- 4. 拼装成目标 JSON 行 ----------
def build_line(conv_str, contact_id):
    """
    构建单行JSON数据
    Args:
        conv_str: 处理后的对话文本内容
        contact_id: 联系人标识符
    Returns:
        格式化的JSON字符串行
    """
    # 构建模型输入输出结构
    row = {
        "input": [
            {"role": "system", "content": prompt_content},  # 系统角色提示内容
            {"role": "user",   "content": conv_str}         # 用户角色对话内容
        ],
        "target": contact_id  # 目标输出为联系人ID
    }
    # 将字典转换为JSON字符串，确保中文字符正常显示
    return json.dumps(row, ensure_ascii=False)
    
# 注册UDF函数，用于在Spark DataFrame中调用
build_line_udf = udf(build_line, StringType())

# 应用UDF函数，将处理后的对话转换为JSON格式
df_json = df_conv.select(
    build_line_udf(col("processed_sentences"), col("contact_id")).alias("json_line")
)

# ---------- 7. 分布式写出单文件 ND-JSON ----------
# 设置输出路径，包含日期标识
tmp_out = "viewfs://hadoop-meituan/user/hadoop-caterb2b-mall-risk-management/rcc/prod/data/dataset/" + f'call_info_{friday_dt}.jsonl'

# 7.1 并行写多 part（无压缩）
# 以覆盖模式将DataFrame内容写入文本文件，每行一个JSON对象
df_json.write.mode("overwrite").text(tmp_out)

In [ ]:
import requests
import json
from pathlib import Path
import pandas as pd
import requests
import time
import json
import os
import openai
from openai import OpenAI
import pandas as pd
from collections import defaultdict
import sys
import os
import pyarrow
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import json

url = "https://friday.sankuai.com/api/friday/ml/v1/experiment/eval/api/task/submit"
taskName = "call" + "_"+ "identify" + "_" + friday_dt
hadoop_path = tmp_out
payload = json.dumps({
   "tenantId":"1666025067693609053", # FRIDAY租户ID
   "taskName":taskName, # 该任务的名称
   "modelName": "DeepSeek-V3", # 模型名称
   "misId": "songxindong", # 提交人的mis号
   "modelParamsObject":{
        "batch_size":"100",
        "max_new_tokens":"500", # 如果调用的是R1，有think的输出，根据实际评估设置，太大也会导致GPU利用率降低
        "prompt_type": "default",  # 使用平台预置规范，需要添加这个参数，对于DeepSeek模型的推理，数据格式推荐使用平台预置规范
        "extra_infer_param": {"temperature": 0.3, "seed": 2025}
    },
   "input": hadoop_path # 替换成业务的待推理数据集
   })
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
taskId = json.loads(response.text)["data"]
code = json.loads(response.text)["code"]
print(response.text)

{"code":0,"data":288509,"message":"成功"}


In [14]:
# 假设 code 是之前定义的变量
if code == 0:
    # 2. 创建表（如不存在）
    spark.sql("""
    CREATE TABLE IF NOT EXISTS mart_caterb2b_mall_risk_management.friday_offline_batch_infer_record (
        task_id STRING COMMENT '任务ID',
        task_name STRING COMMENT '任务名称'
    )
    COMMENT 'firday离线批处理提交记录'
    PARTITIONED BY (dt STRING)
    STORED AS ORC
    """)

    # 3. 写入 taskId
    spark.sql(f"""
      INSERT INTO TABLE mart_caterb2b_mall_risk_management.friday_offline_batch_infer_record PARTITION (dt='{etl_dt}')
      SELECT '{taskId}' AS task_id,
             'callIdentify' AS task_name
    """)
    print(f"taskId {taskId} 已写入 mart_caterb2b_mall_risk_management.friday_offline_batch_infer_record")
else:
    print(f"code = {code}，不执行表创建和数据写入操作")


taskId 288509 已写入 mart_caterb2b_mall_risk_management.friday_offline_batch_infer_record
